In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import json
import math

C:\Users\juan-\AppData\Local\Temp\ipykernel_5280\2784272662.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def rearange_date(date_str):
    if date_str is None: return None
    date_split = date_str.split("/")
    return f"{date_split[2]}-{date_split[1]}-{date_split[0]}"

def rearange_date_2(date_str):
    if date_str is None: return None
    date_split = date_str.split("/")
    return f"{date_split[2]}-{date_split[0]}-{date_split[1]}"

In [59]:
stations_df = pd.read_csv("stations.csv")
stations_df = stations_df.drop('location', axis=1)
stations_df = stations_df.replace({np.nan: None})
stations_df['DataStart'] = stations_df['DataStart'].apply(rearange_date)
stations_df['DataStop'] = stations_df['DataStop'].apply(rearange_date)
stations_df

,IdSensore,NomeTipoSensore,UnitaMisura,Idstazione,NomeStazione,Quota,Provincia,Comune,Storico,DataStart,DataStop,Utm_Nord,UTM_Est,lat,lng
0,10431,Ossidi di Azoto,µg/m³,1264,Sondrio v.Paribelli,290.0,SO,Sondrio,N,2008-11-11,None,5113073,567873,46.167852,9.87921
1,9968,Ossidi di Azoto,µg/m³,679,Valmadrera v.Pozzi,247.0,LC,Valmadrera,N,2006-02-16,None,5076576,527307,45.842207,9.351658
2,5543,Biossido di Azoto,µg/m³,540,Garbagnate Milanese v. Villoresi,175.0,MI,Garbagnate Milanese,S,1992-12-11,2018-07-30,5046467,506105,45.571716,9.078242
3,20197,PM10 (SM2005),µg/m³,598,San Rocco al Porto v.Matteotti,50.0,LO,San Rocco al Porto,N,2002-01-01,None,4992297,555154,45.081987,9.700788
4,5611,Biossido di Azoto,µg/m³,608,Codogno v.Trento,59.0,LO,Codogno,N,2000-01-13,None,5000892,554872,45.159373,9.698148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,5807,Monossido di Carbonio,mg/m³,576,Merate v. Madonna di Loreto,279.0,LC,Merate,S,1991-12-24,None,5060569,531623,45.697952,9.406191
966,5662,Biossido di Zolfo,µg/m³,592,Treviglio p.zza Insurrezione,191.0,BG,Treviglio,S,1991-06-02,None,5040815,546236,45.519335,9.59201
967,10096,Benzene,µg/m³,697,Borgofranco sul Po loc. Bonizzo,14.0,MN,Borgofranco sul Po,S,2006-11-29,2018-01-01,4990430,671752,45.046502,11.180936
968,9924,Monossido di Carbonio,mg/m³,677,Cremona Via Fatebenefratelli,43.0,CR,Cremona,S,2006-04-08,2022-12-31,4999315,582066,45.142541,10.043836


In [60]:
stations_df_2 = pd.read_csv("stations_air_quality_2.csv")
stations_df_2 = stations_df_2.rename(columns={"idSensore": "IdSensore", "UTM_Nord": "Utm_Nord", "IdStazione": "Idstazione"})
stations_df_2['DataStop'] = None
stations_df_2 = stations_df_2.replace({np.nan: None})
stations_df_2['DataStart'] = stations_df_2['DataStart'].apply(rearange_date_2) # Here they
stations_df_2['DataStop'] = stations_df_2['DataStop'].apply(rearange_date_2)
stations_df_2

,IdSensore,NomeTipoSensore,UnitaMisura,Idstazione,NomeStazione,Quota,Provincia,Comune,Storico,DataStart,Utm_Nord,UTM_Est,lat,lng,DataStop
0,5504,Biossido di Azoto,µg/m³,501,Milano v.Marche,129.0,MI,Milano,N,1980-09-18,5038105,514918,45.496319,9.190934,None
1,5507,Biossido di Azoto,µg/m³,504,Sesto San Giovanni v. Cesare da Sesto,139.0,MI,Sesto San Giovanni,N,1980-12-18,5042386,518435,45.534768,9.236109,None
2,5517,Biossido di Azoto,µg/m³,514,Rho via Buon Gesù,152.0,MI,Rho,N,1991-10-03,5041100,503483,45.523429,9.044602,None
3,5520,Biossido di Azoto,µg/m³,517,Turbigo,141.0,MI,Turbigo,N,1990-01-11,5041480,479423,45.526556,8.736502,None
4,5531,Biossido di Azoto,µg/m³,528,Milano Verziere,116.0,MI,Milano,N,1988-11-24,5034443,515270,45.463349,9.195325,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,20574,Biossido di Zolfo,µg/m³,545,Motta Visconti v. De Gasperi,100.0,MI,Motta Visconti,N,None,5014273,499103,45.281964,8.988576,None
229,30158,Particelle sospese PM2.5,µg/m³,649,Brescia v.Broletto,149.0,BS,Brescia,N,2013-12-13,5043674,595467,45.540065,10.22282,None
230,30163,Biossido di Azoto,µg/m³,669,Brescia Villaggio Sereno,70.0,BS,Brescia,N,2013-12-13,5040636,593101,45.513062,10.191906,None
231,30165,Ozono,µg/m³,669,Brescia Villaggio Sereno,70.0,BS,Brescia,N,2013-12-13,5040636,593101,45.513062,10.191906,None


In [61]:
complete_stations = pd.concat([stations_df, stations_df_2], axis=0)
complete_stations

,IdSensore,NomeTipoSensore,UnitaMisura,Idstazione,NomeStazione,Quota,Provincia,Comune,Storico,DataStart,DataStop,Utm_Nord,UTM_Est,lat,lng
0,10431,Ossidi di Azoto,µg/m³,1264,Sondrio v.Paribelli,290.0,SO,Sondrio,N,2008-11-11,None,5113073,567873,46.167852,9.87921
1,9968,Ossidi di Azoto,µg/m³,679,Valmadrera v.Pozzi,247.0,LC,Valmadrera,N,2006-02-16,None,5076576,527307,45.842207,9.351658
2,5543,Biossido di Azoto,µg/m³,540,Garbagnate Milanese v. Villoresi,175.0,MI,Garbagnate Milanese,S,1992-12-11,2018-07-30,5046467,506105,45.571716,9.078242
3,20197,PM10 (SM2005),µg/m³,598,San Rocco al Porto v.Matteotti,50.0,LO,San Rocco al Porto,N,2002-01-01,None,4992297,555154,45.081987,9.700788
4,5611,Biossido di Azoto,µg/m³,608,Codogno v.Trento,59.0,LO,Codogno,N,2000-01-13,None,5000892,554872,45.159373,9.698148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,20574,Biossido di Zolfo,µg/m³,545,Motta Visconti v. De Gasperi,100.0,MI,Motta Visconti,N,None,None,5014273,499103,45.281964,8.988576
229,30158,Particelle sospese PM2.5,µg/m³,649,Brescia v.Broletto,149.0,BS,Brescia,N,2013-12-13,None,5043674,595467,45.540065,10.22282
230,30163,Biossido di Azoto,µg/m³,669,Brescia Villaggio Sereno,70.0,BS,Brescia,N,2013-12-13,None,5040636,593101,45.513062,10.191906
231,30165,Ozono,µg/m³,669,Brescia Villaggio Sereno,70.0,BS,Brescia,N,2013-12-13,None,5040636,593101,45.513062,10.191906


In [62]:
# Remove duplicate stations
print("unique stations: " + str(len(complete_stations.IdSensore.unique())))
complete_stations = complete_stations.drop_duplicates(subset=["IdSensore"]).reset_index(drop=True)
complete_stations

unique stations: 982


,IdSensore,NomeTipoSensore,UnitaMisura,Idstazione,NomeStazione,Quota,Provincia,Comune,Storico,DataStart,DataStop,Utm_Nord,UTM_Est,lat,lng
0,10431,Ossidi di Azoto,µg/m³,1264,Sondrio v.Paribelli,290.0,SO,Sondrio,N,2008-11-11,None,5113073,567873,46.167852,9.87921
1,9968,Ossidi di Azoto,µg/m³,679,Valmadrera v.Pozzi,247.0,LC,Valmadrera,N,2006-02-16,None,5076576,527307,45.842207,9.351658
2,5543,Biossido di Azoto,µg/m³,540,Garbagnate Milanese v. Villoresi,175.0,MI,Garbagnate Milanese,S,1992-12-11,2018-07-30,5046467,506105,45.571716,9.078242
3,20197,PM10 (SM2005),µg/m³,598,San Rocco al Porto v.Matteotti,50.0,LO,San Rocco al Porto,N,2002-01-01,None,4992297,555154,45.081987,9.700788
4,5611,Biossido di Azoto,µg/m³,608,Codogno v.Trento,59.0,LO,Codogno,N,2000-01-13,None,5000892,554872,45.159373,9.698148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,20532,PM10 (SM2005),µg/m³,569,Sondrio v.Mazzini,307.0,SO,Sondrio,N,None,None,5113078,567173,46.167971,9.870146
978,20569,Monossido di Carbonio,mg/m³,1801,Brescia v.Tartaglia,151.0,BS,Brescia,N,2023-01-03,None,5043948,594556,45.542648,10.211205
979,20570,Ozono,µg/m³,659,Odolo v.Praes,349.0,BS,Odolo,N,2023-01-03,None,5055661,608260,45.646052,10.389297
980,20572,PM10 (SM2005),µg/m³,564,Erba v. Battisti,279.0,CO,Erba,N,None,None,5072803,517232,45.808574,9.221779


In [3]:
#complete_stations.to_csv("complete_air_quality_stations.csv")

NameError: name 'complete_stations' is not defined

In [17]:
complete_stations = pd.read_csv("complete_air_quality_stations.csv", index_col=0)
complete_stations = complete_stations.replace({np.nan: None})
complete_stations

,IdSensore,NomeTipoSensore,UnitaMisura,Idstazione,NomeStazione,Quota,Provincia,Comune,Storico,DataStart,DataStop,Utm_Nord,UTM_Est,lat,lng
0,10431,Ossidi di Azoto,µg/m³,1264,Sondrio v.Paribelli,290.0,SO,Sondrio,N,2008-11-11,None,5113073,567873,46.167852,9.87921
1,9968,Ossidi di Azoto,µg/m³,679,Valmadrera v.Pozzi,247.0,LC,Valmadrera,N,2006-02-16,None,5076576,527307,45.842207,9.351658
2,5543,Biossido di Azoto,µg/m³,540,Garbagnate Milanese v. Villoresi,175.0,MI,Garbagnate Milanese,S,1992-12-11,2018-07-30,5046467,506105,45.571716,9.078242
3,20197,PM10 (SM2005),µg/m³,598,San Rocco al Porto v.Matteotti,50.0,LO,San Rocco al Porto,N,2002-01-01,None,4992297,555154,45.081987,9.700788
4,5611,Biossido di Azoto,µg/m³,608,Codogno v.Trento,59.0,LO,Codogno,N,2000-01-13,None,5000892,554872,45.159373,9.698148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,20532,PM10 (SM2005),µg/m³,569,Sondrio v.Mazzini,307.0,SO,Sondrio,N,None,None,5113078,567173,46.167971,9.870146
978,20569,Monossido di Carbonio,mg/m³,1801,Brescia v.Tartaglia,151.0,BS,Brescia,N,2023-01-03,None,5043948,594556,45.542648,10.211205
979,20570,Ozono,µg/m³,659,Odolo v.Praes,349.0,BS,Odolo,N,2023-01-03,None,5055661,608260,45.646052,10.389297
980,20572,PM10 (SM2005),µg/m³,564,Erba v. Battisti,279.0,CO,Erba,N,None,None,5072803,517232,45.808574,9.221779


In [18]:
base_url = "http://127.0.0.1:8000/api/collections/airqualitysensor/"
body = {
    "bulk": True,
    "items": []
}

station_to_add = complete_stations.copy()
for i, sensor in station_to_add.iterrows():
    item = {
        "sensor_id": sensor.IdSensore,
        "sensor_type": sensor.NomeTipoSensore,
        "measurement_unit": sensor.UnitaMisura,
        "station_id": sensor.Idstazione,
        "station_name": sensor.NomeStazione,
        "altitude": sensor.Quota,
        "province": sensor.Provincia,
        "comune": sensor.Comune,
        "is_historical": sensor.Storico == 'S',
        "date_start": sensor.DataStart, #YYYY-MM-DD
        "date_stop": sensor.DataStop, #YYYY-MM-DD
        "utm_north": sensor.Utm_Nord,
        "utm_east": sensor.UTM_Est,
        "latitude": sensor.lat,
        "longitude": sensor.lng,
        "location": f"POINT ({sensor.lng} {sensor.lat})"
    }
    body["items"].append(item)
    
body_json = json.dumps(body)

#uncomment for creating the stations
req = requests.post(base_url, body_json)
if req.status_code == 500:
    print(body_json)
print(f'request {i} - {req.status_code}')


request 981 - 200


In [64]:
complete_stations.loc[complete_stations["IdSensore"] == 20565]

,IdSensore,NomeTipoSensore,UnitaMisura,Idstazione,NomeStazione,Quota,Provincia,Comune,Storico,DataStart,DataStop,Utm_Nord,UTM_Est,lat,lng
